In [9]:
import pandas as pd
from gurobipy import *

#input parameters
FILE_PATH = "jeroen_punt_nl_dynamische_stroomprijzen_jaar_2024.csv"
FIXED_PRICE = 0.289  # €/kWh (fixed contract incl. taxes)
POWER_KW = 7.4       # charging power (1-phase wallbox)
CHARGE_START = 18    # start hour
CHARGE_END = 7       # end hour
TAX_SURCHARGE = 0.10880
BTW = 1.21

#data set
df = pd.read_csv(FILE_PATH, sep=";", decimal=",")
df["datum_nl"] = pd.to_datetime(df["datum_nl"])
df["hour"] = df["datum_nl"].dt.hour
df["weekday"] = df["datum_nl"].dt.day_name()

# apply btw and taxes
df["prijs_incl_belastingen"] = (df["prijs_excl_belastingen"] + TAX_SURCHARGE) * BTW


# function per scenario
def calculate_scenario(name, active_days, energy_kwh_per_day):
    needed_hours = int(energy_kwh_per_day / POWER_KW)

    # Select charging window between 18:00 and 07:00
    df_charge = df[(df["hour"] >= CHARGE_START) | (df["hour"] < CHARGE_END)].copy()

    # Keep only active days 
    if active_days:
        df_charge = df_charge[df_charge["weekday"].isin(active_days)]

    # Select the first hours charging immediatly
    all_days = sorted(df_charge["datum_nl"].dt.date.unique())

    charged_rows = []
    optimized_daily_costs = []

    for i in range(len(all_days) - 1):
        today = all_days[i]
        tomorrow = all_days[i + 1]

        # Choosing time window between 18 and 07 next day
        window_today = df[(df["datum_nl"].dt.date == today) & (df["hour"] >= CHARGE_START)]
        window_next = df[(df["datum_nl"].dt.date == tomorrow) & (df["hour"] < CHARGE_END)]
        window = pd.concat([window_today, window_next]).sort_values("datum_nl")

        # charge immediately (non-optimized)
        charged_rows.append(window.iloc[:needed_hours])

        # optimization model
        timesteps = range(len(window))
        prices = {t: window.iloc[t]["prijs_incl_belastingen"] for t in timesteps}

        model = Model()
        charge = model.addVars(timesteps, lb=0, ub=POWER_KW)  # adding decision variable

        # The total amount charged needs to be the same as the amount of energy needed
        model.addConstr(sum(charge[t] for t in timesteps) == energy_kwh_per_day)

        # Objective: minimize cost
        model.setObjective(sum(prices[t] * charge[t] for t in timesteps), GRB.MINIMIZE)
        model.setParam("OutputFlag", 0)
        model.optimize()

        optimized_daily_costs.append(model.objVal)

    # Combine all non-optimized charging rows
    df_nonopt = pd.concat(charged_rows)

    # Calculate non-optimized costs
    df_nonopt["cost_dynamic"] = df_nonopt["prijs_incl_belastingen"] * POWER_KW
    df_nonopt["cost_fixed"] = FIXED_PRICE * POWER_KW

    daily_costs = df_nonopt.groupby(df_nonopt["datum_nl"].dt.date)[["cost_dynamic", "cost_fixed"]].sum()
    total_dynamic = daily_costs["cost_dynamic"].sum()
    total_fixed = daily_costs["cost_fixed"].sum()

    # Calculate optimized totals
    total_optimized = sum(optimized_daily_costs)
    diff_opt_vs_fixed = total_optimized - total_fixed
    diff_opt_vs_dyn = total_optimized - total_dynamic

    return {
        "Scenario": name,
        "Active days": ", ".join(active_days) if active_days else "Every day",
        "Energy/day (kWh)": energy_kwh_per_day,
        "Total dynamic (€)": total_dynamic,
        "Total fixed (€)": total_fixed,
        "Total optimized (€)": total_optimized,
        "Opt - fixed (€)": diff_opt_vs_fixed,
        "Opt - dyn (€)": diff_opt_vs_dyn
    }


scenarios = [
    ("Distant commuter", [], 29.6),                      # daily
    ("City hopper", [], 7.4),                            # daily
    ("Road tripper", ["Saturday", "Sunday"], 29.6),      # weekends
    ("Grocery grabber", ["Wednesday", "Sunday"], 7.4)    # twice per week
]

# results
results = [calculate_scenario(*s) for s in scenarios]
df_results = pd.DataFrame(results)

# save file
df_results.to_csv("scenario_costs_2024_optimized.csv", index=False)
print("Results saved to 'scenario_costs_2024_optimized.csv'")

# results printed
print(f"\n{'Scenario':<20} {'Active days':<25} {'Energy/day (kWh)':>17} "
      f"{'Total dynamic (€)':>20} {'Total fixed (€)':>20} {'Total optimized (€)':>23} "
      f"{'Opt - fixed (€)':>20} {'Opt - dyn (€)':>20}")
print("-" * 170)

for r in results:
    print(f"{r['Scenario']:<20} {r['Active days']:<25} {r['Energy/day (kWh)']:>17.1f} "
          f"{r['Total dynamic (€)']:>20.2f} {r['Total fixed (€)']:>20.2f} {r['Total optimized (€)']:>23.2f} "
          f"{r['Opt - fixed (€)']:>20.2f} {r['Opt - dyn (€)']:>20.2f}")


Results saved to 'scenario_costs_2024_optimized.csv'

Scenario             Active days                Energy/day (kWh)    Total dynamic (€)      Total fixed (€)     Total optimized (€)      Opt - fixed (€)        Opt - dyn (€)
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Distant commuter     Every day                              29.6              2865.14              3122.36                 2256.47              -865.89              -608.67
City hopper          Every day                               7.4               710.26               780.59                  550.45              -230.14              -159.80
Road tripper         Saturday, Sunday                       29.6               742.95               881.10                  610.20              -270.90              -132.75
Grocery grabber      Wednesday, Sunday                       7.4               196.